In [5]:
import mosek as mo
import numpy as np
import sys

In [3]:
# Expected returns and covariance matrix
m = np.array(
    [0.0720, 0.1552, 0.1754, 0.0898, 0.4290, 0.3929, 0.3217, 0.1838]
 )
S = np.array([
     [0.0946, 0.0374, 0.0349, 0.0348, 0.0542, 0.0368, 0.0321, 0.0327],
     [0.0374, 0.0775, 0.0387, 0.0367, 0.0382, 0.0363, 0.0356, 0.0342],
     [0.0349, 0.0387, 0.0624, 0.0336, 0.0395, 0.0369, 0.0338, 0.0243],
     [0.0348, 0.0367, 0.0336, 0.0682, 0.0402, 0.0335, 0.0436, 0.0371],
     [0.0542, 0.0382, 0.0395, 0.0402, 0.1724, 0.0789, 0.0700, 0.0501],
     [0.0368, 0.0363, 0.0369, 0.0335, 0.0789, 0.0909, 0.0536, 0.0449],
     [0.0321, 0.0356, 0.0338, 0.0436, 0.0700, 0.0536, 0.0965, 0.0442],
     [0.0327, 0.0342, 0.0243, 0.0371, 0.0501, 0.0449, 0.0442, 0.0816]
 ])

In [11]:
N = m.shape[0]  # Number of securities
gamma2 = 0.05  # Risk limit

# Cholesky factor of S to use in conic risk constraint
G = np.linalg.cholesky(S)
print(G)


[[0.30757113 0.         0.         0.         0.         0.
  0.         0.        ]
 [0.12159789 0.25042754 0.         0.         0.         0.
  0.         0.        ]
 [0.11346969 0.09943924 0.19908909 0.         0.         0.
  0.         0.        ]
 [0.11314456 0.09161077 0.05852564 0.20875949 0.         0.
  0.         0.        ]
 [0.1762194  0.06697384 0.06451691 0.0495801  0.36088864 0.
  0.         0.        ]
 [0.11964712 0.08685611 0.07376985 0.03682801 0.12583779 0.21542316
  0.         0.        ]
 [0.1043661  0.09148076 0.06459838 0.09403289 0.10156042 0.05644071
  0.2252194  0.        ]
 [0.10631687 0.08494311 0.01903459 0.07748203 0.05709883 0.06201184
  0.03338534 0.22021645]]


In [6]:
# Create a MOSEK model
with mo.Model("markowitz") as M:
     # Settings
     M.setLogHandler(sys.stdout)

     # Decision variable (fraction of holdings in each security)
     # The variable x is the fraction of holdings in each security.
     # x must be positive, this imposes the no short-selling constraint.
     x = M.variable("x", N, mo.Domain.greaterThan(0.0))

     # Budget constraint
     M.constraint('budget', mo.Expr.sum(x), mo.Domain.equalsTo(1))

     # Objective
     M.objective('obj', mo.ObjectiveSense.Maximize, mo.Expr.dot(m, x))

     # Imposes a bound on the risk
     M.constraint('risk', mo.Expr.vstack(gamma2, 0.5, mo.Expr.mul(G.T, x)),
                  mo.Domain.inRotatedQCone())

     # Solve optimization
     M.solve()

     returns = M.primalObjValue()
     portfolio = x.level()

AttributeError: module 'mosek' has no attribute 'Model'